# تحليل الأثر المؤقت وخوارزمية التنفيذ المثلى

هذا الدفتر يهدف إلى:
- تحليل نموذج الأثر المؤقت للصفقات $g_t(x)$ باستخدام بيانات حقيقية لثلاثة رموز (CRWV, FROG, SOUN) مع مقارنة بين النماذج الخطية وغير الخطية.
- بناء إطار رياضي وخوارزمية تقريبية للتنفيذ الأمثل بحيث $\sum_i x_i = S$.

ستجد في الدفتر:
1. استيراد وتجهيز البيانات.
2. استكشاف وتحليل بصري للبيانات.
3. ملاءمة النماذج الرياضية ومقارنة النتائج.
4. شرح رياضي لخوارزمية التنفيذ.
5. كود تطبيقي لخوارزمية التنفيذ المثلى.

> **ملاحظة:** يجب توفر ملفات البيانات الأصلية في المسارات المحددة.

In [ ]:
# تثبيت matplotlib إذا لم تكن مثبتة
%pip install matplotlib

# استيراد المكتبات الأساسية للتحليل والرسم البياني
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import os

# إعداد الرسوم البيانية لتكون أكثر وضوحًا وجمالًا
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
# تحميل البيانات الحقيقية من ملفات zip لكل رمز مع معالجة الأخطاء

ticker_paths = {
    'CRWV': 'data/CRWV/CRWV.zip',
    'FROG': 'data/FROG/FROG.zip',
    'SOUN': 'data/SOUN/SOUN.zip'
}

ticker_dfs = {}

for ticker, zip_path in ticker_paths.items():
    print(f'\n--- معالجة بيانات {ticker} ---')
    if not os.path.exists(zip_path):
        print(f'❌ الملف غير موجود: {zip_path}')
        continue
    try:
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(f'data/{ticker}/extracted')
            csv_files = [f for f in os.listdir(f'data/{ticker}/extracted') if f.endswith(".csv")]
            if not csv_files:
                print(f'❌ لا يوجد ملف CSV داخل {zip_path}')
                continue
            try:
                df = pd.read_csv(f'data/{ticker}/extracted/' + csv_files[0])
            except Exception as e:
                print(f'❌ خطأ في قراءة CSV: {e}')
                continue
            required_cols = set(['price', 'volume'])
            if not required_cols.issubset(df.columns):
                print(f'❌ ملف {csv_files[0]} لا يحتوي الأعمدة المطلوبة: {required_cols - set(df.columns)}')
                continue
            ticker_dfs[ticker] = df
            print(f'✅ تم تحميل البيانات ({len(df)}) صف')
            display(df.head(5))
    except Exception as e:
        print(f'❌ خطأ أثناء معالجة {ticker}: {e}')

if not ticker_dfs:
    print('⚠️ لم يتم تحميل أي بيانات. تأكد من وجود الملفات بالمسارات الصحيحة وبالصيغة المطلوبة.')

In [1]:
# استكشاف وتحليل بصري للبيانات

for ticker, df in ticker_dfs.items():
    print(f'\n--- تحليل {ticker} ---')
    display(df.describe().T)
    if 'price' in df.columns and 'volume' in df.columns:
        fig, axes = plt.subplots(1, 2, figsize=(14, 4))
        sns.lineplot(ax=axes[0], data=df['price'], color='royalblue')
        axes[0].set_title(f'سعر {ticker}')
        axes[0].set_xlabel('الزمن')
        axes[0].set_ylabel('السعر')
        sns.lineplot(ax=axes[1], data=df['volume'], color='darkorange')
        axes[1].set_title(f'حجم التداول {ticker}')
        axes[1].set_xlabel('الزمن')
        axes[1].set_ylabel('الحجم')
        plt.suptitle(f'تحليل بياني لـ {ticker}', fontsize=14)
        plt.tight_layout()
        plt.show()

NameError: name 'ticker_dfs' is not defined

## Modeling the Temporary Impact $g_t(x)$

The temporary market impact function $g_t(x)$ models the immediate price change caused by trading $x$ shares at time $t$. A common approach is to assume a linear relationship: $g_t(x) \approx \beta_t x$. However, this can be a gross oversimplification, as real market impact often exhibits nonlinearities due to liquidity, order book depth, and market microstructure effects.

### Linear Model
- **Form:** $g_t(x) = \beta_t x$
- **Pros:** Simple, interpretable, easy to estimate.
- **Cons:** May not capture diminishing/increasing returns to scale, ignores nonlinear liquidity effects.

### Nonlinear Model
- **Form:** $g_t(x) = \alpha_t x^p$ (with $p \neq 1$)
- **Pros:** Can capture concave/convex impact, more flexible.
- **Cons:** More complex to estimate, risk of overfitting with limited data.

We will fit both models to the data and compare their fit.

In [ ]:
%pip install scipy

# ملاءمة النماذج الخطية وغير الخطية للأثر المؤقت g_t(x)
from scipy.optimize import curve_fit

def linear_model(x, beta):
    return beta * x

def nonlinear_model(x, alpha, p):
    return alpha * np.power(x, p)

fit_results = {}

for ticker, df in ticker_dfs.items():
    if 'price' in df.columns and 'volume' in df.columns:
        print(f'\n--- ملاءمة النماذج لـ {ticker} ---')
        df = df.copy()
        df['impact'] = df['price'].diff().fillna(0)
        x = df['volume'].values
        y = df['impact'].values
        # ملاءمة النموذج الخطي
        try:
            popt_lin, _ = curve_fit(linear_model, x, y)
        except Exception as e:
            print(f'خطأ في النموذج الخطي: {e}')
            continue
        # ملاءمة النموذج غير الخطي
        try:
            popt_nonlin, _ = curve_fit(nonlinear_model, x, y, bounds=([0,0],[np.inf,2]))
        except Exception as e:
            print(f'خطأ في النموذج غير الخطي: {e}')
            continue
        fit_results[ticker] = {'linear': popt_lin, 'nonlinear': popt_nonlin}
        # رسم النتائج
        plt.figure(figsize=(7,4))
        plt.scatter(x, y, alpha=0.3, label='البيانات')
        x_fit = np.linspace(x.min(), x.max(), 100)
        plt.plot(x_fit, linear_model(x_fit, *popt_lin), label='النموذج الخطي', color='royalblue')
        plt.plot(x_fit, nonlinear_model(x_fit, *popt_nonlin), label='النموذج غير الخطي', color='darkorange')
        plt.title(f'ملاءمة الأثر المؤقت لـ {ticker}')
        plt.xlabel('الحجم')
        plt.ylabel('تغير السعر')
        plt.legend()
        plt.show()
        print(f'معامل النموذج الخطي (beta): {popt_lin[0]:.4g}')
        print(f'معاملات النموذج غير الخطي (alpha, p): {popt_nonlin[0]:.4g}, {popt_nonlin[1]:.4g}')

### مناقشة ونتائج النماذج

النموذج الخطي $g_t(x) = \beta_t x$ بسيط وسهل التفسير، لكنه غالبًا لا يعكس الواقع بدقة في الأسواق ذات السيولة المنخفضة أو عند تنفيذ أوامر كبيرة. النموذج غير الخطي $g_t(x) = \alpha_t x^p$ يوفر مرونة أكبر ويعكس سلوك السوق بشكل أفضل عند وجود لاخطية في التأثير.

**استنتاج:**
- استخدم النموذج غير الخطي إذا أظهرت البيانات أن $p$ يختلف عن 1 بشكل واضح.
- في حال تشابه النتائج، يمكن الاكتفاء بالنموذج الخطي للبساطته.

> **توصية:** راقب دائمًا جودة الملاءمة وفسر النتائج بناءً على خصائص السوق والبيانات المتوفرة.

### الإطار الرياضي وخوارزمية التنفيذ المثلى

**المطلوب:** تنفيذ كمية كلية $S$ من الأسهم على $N$ فترات بحيث $\sum x_i = S$ مع تقليل التكلفة الكلية التي تشمل الأثر المؤقت.

**الدالة الهدف:**

$$
\min_{x_1,...,x_N} \sum_{i=1}^N [P_{t_i} x_i + g_i(x_i)]
$$

**القيود:**
- $\sum_{i=1}^N x_i = S$
- $x_i \geq 0$

**خطوات الحل:**
1. إذا كان $g_i(x)$ خطيًا، الحل تحليلي (Almgren-Chriss).
2. إذا كان غير خطي، استخدم خوارزميات عددية مثل `scipy.optimize.minimize`.
3. تحقق من تحقق القيود بعد الحل.

> **ملاحظة:** يمكن تعديل الدالة الهدف أو القيود حسب متطلبات السوق أو المخاطر.

In [ ]:
# Example: Optimal Execution with Nonlinear Impact
from scipy.optimize import minimize

S = 10000  # total shares
N = 10     # intervals
alpha = 0.01
p = 0.8
P0 = 100

def cost(x):
    x = np.array(x)
    return np.sum(P0 * x + alpha * np.power(x, p))

# Constraint: sum(x) = S
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - S})
# Bounds: x_i >= 0
bounds = [(0, None)] * N
x0 = np.ones(N) * (S / N)

res = minimize(cost, x0, bounds=bounds, constraints=cons)
print('Optimal x:', res.x)
print('Total cost:', res.fun)